[🥭 Entropy Markets](https://entropy.trade/) support is available at: [Docs](https://docs.entropy.trade/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmakerSpot.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmakerSpot.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Practical: Run Marketmaker On Spot

This notebook shows is very similar to the regular [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb). The only difference is this one runs on a Spot market instead of a Perp market.

It contains the following sections:
1. This section contains all the code that is identical to the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb).
2. This section contains the code that is new to this notebook, specifically for working with a Spot market. It contains one method - `build_spot_model_state()` - which loads Spot market data.
3. This final section runs the marketmaker.

## 1. Identical Code

This code is all taken from the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) with no changes. If you followed along with that Practical then you have already seen all this code.

In [ ]:
import logging
import entropy
import entropy.marketmaking
import time

from datetime import timedelta
from decimal import Decimal

from entropy.marketmaking.orderchain.chain import Chain
from entropy.marketmaking.orderchain.biasquoteonpositionelement import BiasQuoteOnPositionElement
from entropy.marketmaking.orderchain.ratioselement import RatiosElement
from entropy.marketmaking.orderchain.roundtolotsizeelement import RoundToLotSizeElement

# Set logging so messages show up.
logging.getLogger().setLevel(logging.INFO)


def build_context():
    return entropy.ContextBuilder.build(cluster_name="mainnet")


def build_order_chain():
    order_type = entropy.OrderType.POST_ONLY
    spread = Decimal("0.005")  # 0.5%
    position_size = Decimal("0.05")  # 5%
    bias = [Decimal("0.0001")]

    ratios_element = RatiosElement(order_type, None, 20, [spread], [position_size], False)
    bias_element = BiasQuoteOnPositionElement(bias)
    round_to_lot_size_element = RoundToLotSizeElement()

    chain = Chain([ratios_element, bias_element, round_to_lot_size_element])

    return chain


def build_order_reconciler():
    return entropy.marketmaking.ToleranceOrderReconciler(Decimal("0.001"), Decimal("0.001"), timedelta(seconds=5))


def build_oracle(context, market):
    oracle_source_name = "pyth"
    oracle_provider = entropy.create_oracle_provider(context, oracle_source_name)
    oracle = oracle_provider.oracle_for_market(context, market)
    if oracle is None:
        raise Exception(f"Could not find oracle for market {market.symbol} from provider {oracle_source_name}.")
    return oracle


def build_marketmaker(context, wallet, account, market):
    chain = build_order_chain()
    order_reconciler = build_order_reconciler()
    instruction_builder = entropy.instruction_builder(context, wallet, account, market.symbol, False)
    market_maker = entropy.marketmaking.MarketMaker(wallet, market, instruction_builder, chain, order_reconciler, Decimal(0))
    return market_maker


## 2. Build the Spot `ModelState`

In the previous [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) the `build_model_state()` function handled building the `ModelState` for a Perp market.

There are some important differences between Spot markets and Perp markets, so this section defines a `build_spot_model_state()` function to perform the equivalent actions to build the `ModelState` for a Spot market.

In [ ]:
def build_spot_model_state(context, account, market, oracle, group_address, cache_address):
    # Build a list of addresses and use `load_multiple()` to fetch them all in one go.
    addresses = [
        group_address,
        cache_address,
        account.address,
        market.bids_address,
        market.asks_address,
        market.event_queue_address,
        *list([oo for oo in account.spot_open_orders if oo is not None])
    ]
    account_infos = entropy.AccountInfo.load_multiple(context, addresses)
    group = entropy.Group.parse_with_context(context, account_infos[0])
    cache = entropy.Cache.parse(account_infos[1])
    account = entropy.Account.parse(account_infos[2], group, cache)

    spot_open_orders_account_infos_by_address = {
        str(account_info.address): account_info for account_info in account_infos[6:]}

    all_open_orders = {}
    for slot in account.slots:
        if slot.spot_open_orders is not None and str(slot.spot_open_orders) in spot_open_orders_account_infos_by_address:
            account_info: entropy.AccountInfo = spot_open_orders_account_infos_by_address[str(slot.spot_open_orders)]
            open_orders: entropy.OpenOrders = entropy.OpenOrders.parse(
                account_info,
                slot.base_token_bank.token,
                account.shared_quote_token)
            all_open_orders[str(slot.spot_open_orders)] = open_orders

    group_slot = group.slot_by_spot_market_address(market.address)
    open_orders_address = account.spot_open_orders_by_index[group_slot.index]
    placed_orders_container: entropy.PlacedOrdersContainer = all_open_orders[str(open_orders_address)]

    # Spot markets don't accrue MNGO liquidity incentives
    mngo = entropy.token(context, "MNGO")
    mngo_accrued = entropy.InstrumentValue(mngo, Decimal(0))

    base_value = entropy.InstrumentValue.find_by_symbol(account.net_values, market.base.symbol)
    quote_value = account.shared_quote.net_value

    event_queue = entropy.SerumEventQueue.parse(account_infos[5])

    frame = account.to_dataframe(group, all_open_orders, cache)
    available_collateral = account.init_health(frame)
    inventory = entropy.Inventory(entropy.InventorySource.ACCOUNT,
                                mngo_accrued,
                                available_collateral,
                                base_value,
                                quote_value)

    orderbook = market.parse_account_infos_to_orderbook(account_infos[3], account_infos[4])

    price = oracle.fetch_price(context)

    return entropy.ModelState(open_orders_address,
                            market,
                            entropy.ManualUpdateWatcher(group),
                            entropy.ManualUpdateWatcher(account),
                            entropy.ManualUpdateWatcher(price),
                            entropy.ManualUpdateWatcher(placed_orders_container),
                            entropy.ManualUpdateWatcher(inventory),
                            entropy.ManualUpdateWatcher(orderbook),
                            entropy.ManualUpdateWatcher(event_queue))

## 8. Run the `MarketMaker`

This code is also _nearly_ identical to the [Practical: Run Marketmaker example](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb). The differences are:

* It loads market SOL/USDC instead of SOL-PERP
* It uses a `SpotCollateralCalculator` instead of a `PerpCollateralCalculator`
* It calls our new `build_spot_model_state()` to build the `ModelState`

### Caveat

Please bear in mind that the code below uses a shared Solana `Keypair` and a shared Entropy `Account`. If you're going to run this example more than once or twice, it might be better for you to set up a fresh mainnet `Keypair` and `Account` for you to use - it's easy and free, and it means other people running this example won't accidentally cancel your orders or cause confusion.

In [ ]:
# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

# Specify the market we're going to use
market_symbol = "SOL/USDC"

# Configure how long to pause between pulses
pause_seconds = 30

# Create a 'mainnet' Context
context = build_context()

# Load the wallet's account
group = entropy.Group.load(context)
accounts = entropy.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Load the market
market = entropy.market(context, market_symbol)

oracle = build_oracle(context, market)
marketmaker = build_marketmaker(context, wallet, account, market)

stop_requested = False
while not stop_requested:
    try:
        model_state = build_spot_model_state(context, account, market, oracle, group.address, group.cache)
        marketmaker.pulse(context, model_state)

        # Wait and hope for fills.
        print(f"Pausing for {pause_seconds} seconds.\n")
        time.sleep(pause_seconds)
    except KeyboardInterrupt:
        stop_requested = True
    except Exception as exception:
        print(f"Continuing after problem running market-making iteration: {exception}")

market_operations = entropy.operations(context, wallet, account, market.symbol, dry_run=False)
for order in market_operations.load_my_orders():
    market_operations.cancel_order(order, ok_if_missing=True)

context.dispose()
print("Example complete")
